###INSTALLING DEPENDENCIES

In [1]:
!pip instal -q numpy==1.26.4 pandas==2.2.2 torch==2.4.1+cu121 torchaudio==2.4.1+cu121 scitkit-learn==1.5.2 tqdm==4.66.5

ERROR: unknown command "instal" - maybe you meant "install"


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import warnings
import os
import torch
import torchaudio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
from torchsummary import summary
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import os
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

Seed

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

###CONECTING TO DRIVE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##READING CSV

In [5]:
train = pd.read_csv('/content/Train (1).csv')
test = pd.read_csv('/content/Test_1 (1).csv')

train.head()

,id,audio_filepath,duration,class
0,id_6nazjwygdl,id_6nazjwygdl.wav,1.984,left
1,id_jpwl97yilf,id_jpwl97yilf.wav,2.027,down
2,id_9d131fb32p,id_9d131fb32p.wav,2.274,go
3,id_kat2oty4s0,id_kat2oty4s0.wav,2.176,down
4,id_g3hvae3jbe,id_g3hvae3jbe.wav,1.634,down


In [6]:
test.head()

,id,audio_filepath,duration
0,id_u5iqtgjzhx,id_u5iqtgjzhx.wav,2.35800
1,id_l7ebzcfk5e,id_l7ebzcfk5e.wav,3.22100
2,id_jbzci8uepl,id_jbzci8uepl.wav,2.94898
3,id_jzil0fw5vs,id_jzil0fw5vs.wav,4.33100
4,id_o7mrvf5wj7,id_o7mrvf5wj7.wav,1.85600


In [7]:
test["duration"].max()

7.765

###LOADING DATA

In [ ]:
class OptimizedAudioDataset(Dataset):
    def __init__(self, csv_file, audio_dir, transform=None, max_length=22050*8, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.audio_dir = audio_dir
        self.transform = transform
        self.max_length = max_length
        self.is_test = is_test

        if not self.is_test:
            self.label_encoder = LabelEncoder()
            self.data['class'] = self.label_encoder.fit_transform(self.data['class'].fillna(''))

        # Pre-load all audio files
        self.audio_data = self.preload_audio()

    def preload_audio(self):
        audio_data = {}
        for _, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Preloading audio"):
            audio_path = os.path.join(self.audio_dir, row['audio_filepath'])
            waveform, _ = torchaudio.load(audio_path)
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            if waveform.shape[1] < self.max_length:
                padding = torch.zeros(1, self.max_length - waveform.shape[1])
                waveform = torch.cat([waveform, padding], dim=1)
            else:
                waveform = waveform[:, :self.max_length]
            audio_data[row['audio_filepath']] = waveform
        return audio_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_filepath = self.data.iloc[idx]['audio_filepath']
        waveform = self.audio_data[audio_filepath]

        if self.transform:
            waveform = self.transform(waveform)

        if self.is_test:
            return waveform
        else:
            label = self.data.iloc[idx]['class']
            return waveform, label

def optimized_audio_transform(waveform, sample_rate=22050, n_mels=64, n_fft=400, hop_length=160):
    # Apply pre-emphasis filter
    waveform = torchaudio.functional.preemphasis(waveform)

    # Compute mel spectrogram
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_mels=n_mels,
        n_fft=n_fft,
        hop_length=hop_length
    )(waveform)

    # Convert to decibel scale
    mel_spectrogram = torchaudio.transforms.AmplitudeToDB()(mel_spectrogram)

    return mel_spectrogram.squeeze(0).transpose(0, 1)  # Reshape to (time_steps, n_mels)

def custom_collate_fn(batch):
    if isinstance(batch[0], tuple):
        waveforms, labels = zip(*batch)
        waveforms = torch.stack(waveforms)
        return waveforms, torch.tensor(labels)
    else:
        return torch.stack(batch)

train_dataset = OptimizedAudioDataset(csv_file="/content/Train (1).csv",
                                      audio_dir="/content/drive/MyDrive/Cleaned_TechCabal_01",
                                      transform=optimized_audio_transform)

train_data, val_data = train_test_split(train_dataset, test_size=0.1, random_state=42,
                                        stratify=train_dataset.data['class'])

train_loader = DataLoader(train_data, batch_size=100, shuffle=True,
                          collate_fn=custom_collate_fn, num_workers=4)

val_loader = DataLoader(val_data, batch_size=100, shuffle=False,
                        collate_fn=custom_collate_fn, num_workers=4)

test_dataset = OptimizedAudioDataset(csv_file="/content/Test_1 (1).csv",
                                     audio_dir="/content/drive/MyDrive/Cleaned_TechCabal_01",
                                     transform=optimized_audio_transform,
                                     is_test=True)

test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False,
                         collate_fn=custom_collate_fn, num_workers=4)

Preloading audio: 100%|██████████| 2946/2946 [13:48<00:00,  3.56it/s]


###MODEL

In [8]:
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation block for channel-wise attention
    """
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class TCSConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(TCSConv, self).__init__()
        self.depthwise_conv = nn.Conv1d(in_channels, in_channels, kernel_size, groups=in_channels, padding='same')
        self.pointwise_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.se = SEBlock(out_channels)

    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = self.se(x)
        return x

class SubBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(SubBlock, self).__init__()
        self.tcs_conv = TCSConv(in_channels, out_channels, kernel_size)
        self.bnorm = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(p=0.1)
        self.activation = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.tcs_conv(x)
        x = self.bnorm(x)
        x = self.activation(x)
        x = self.dropout(x)
        # Add residual connection if input and output channels match
        if x.shape == residual.shape:
            x += residual
        return x

class MainBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, R=1):
        super(MainBlock, self).__init__()
        self.residual_pointwise = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.residual_batchnorm = nn.BatchNorm1d(out_channels)

        self.sub_blocks = nn.ModuleList([
            SubBlock(in_channels if i == 0 else out_channels, out_channels, kernel_size)
            for i in range(R)
        ])

    def forward(self, x):
        residual = self.residual_batchnorm(self.residual_pointwise(x))
        for layer in self.sub_blocks:
            x = layer(x)
        return x + residual  # Add the residual connection at the end of the MainBlock

class MatchboxNet(nn.Module):
    def __init__(self, B, R, C, kernel_sizes=None, NUM_CLASSES=30):
        super(MatchboxNet, self).__init__()
        if not kernel_sizes:
            kernel_sizes = [k*2+11 for k in range(1, B+1)]

        self.prologue = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=11, stride=2),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        self.blocks = nn.ModuleList([
            MainBlock(128 if i == 0 else C, C, kernel_size=kernel_sizes[i], R=R)
            for i in range(B)
        ])

        self.epilogue = nn.Sequential(
            nn.Conv1d(C, 128, kernel_size=29, dilation=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, NUM_CLASSES, kernel_size=1),
            nn.AdaptiveAvgPool1d(1)
        )

    def forward(self, x):
        x = self.prologue(x)
        for block in self.blocks:
            x = block(x)
        x = self.epilogue(x).squeeze(2)
        return F.log_softmax(x, dim=1)

###CHECKING MODEL SUMMARY

In [9]:
model = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=8).to("cuda")
summary(model, input_size=(64, 438))  # Adjust input_size to match (batch_size, input_dim)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 128, 214]          90,240
       BatchNorm1d-2             [-1, 128, 214]             256
              ReLU-3             [-1, 128, 214]               0
            Conv1d-4              [-1, 64, 214]           8,256
       BatchNorm1d-5              [-1, 64, 214]             128
            Conv1d-6             [-1, 128, 214]           1,792
            Conv1d-7              [-1, 64, 214]           8,256
 AdaptiveAvgPool1d-8                [-1, 64, 1]               0
            Linear-9                    [-1, 4]             256
             ReLU-10                    [-1, 4]               0
           Linear-11                   [-1, 64]             256
          Sigmoid-12                   [-1, 64]               0
          SEBlock-13              [-1, 64, 214]               0
          TCSConv-14              [-1, 

###TRAINING

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=5, grad_clip=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using..",device)
    model.to(device)
    steps_per_epoch = len(train_loader)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience, verbose=True)
   # scheduler = OneCycleLR(
    #optimizer,
  #  max_lr=0.01,  # Maximum learning rate
   # steps_per_epoch=steps_per_epoch,
  #  epochs=num_epochs,
   # pct_start=0.3,  # Percentage of the cycle to increase learning rate
   # anneal_strategy='cos',  # Annealing strategy ('linear' or 'cos')
   # div_factor=25.0,  # Initial learning rate = max_lr / div_factor
   # final_div_factor=1e4  # Minimum learning rate = initial_lr / final_div_factor
#)

    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.permute(0, 2, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                inputs = inputs.permute(0, 2, 1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("--------------------")

        # Adjust learning rate based on validation loss
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), '/content/drive/MyDrive/MatchboxNet_best_model1.pt')

    print("Training completed.")

num_classes = len(train_dataset.label_encoder.classes_)
model = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

Using.. cuda
Epoch [1/90] Train Loss: 1.8614, Train Acc: 0.3306 Val Loss: 11.9443, Val Acc: 0.1217
--------------------
Epoch [2/90] Train Loss: 1.2452, Train Acc: 0.6681 Val Loss: 1.3352, Val Acc: 0.5300
--------------------
Epoch [3/90] Train Loss: 0.7912, Train Acc: 0.8119 Val Loss: 10.4122, Val Acc: 0.1217
--------------------
Epoch [4/90] Train Loss: 0.5435, Train Acc: 0.8665 Val Loss: 4.2066, Val Acc: 0.2884
--------------------
Epoch [5/90] Train Loss: 0.3883, Train Acc: 0.9004 Val Loss: 1.1756, Val Acc: 0.5880
--------------------
Epoch [6/90] Train Loss: 0.3018, Train Acc: 0.9152 Val Loss: 3.8960, Val Acc: 0.2303
--------------------
Epoch [7/90] Train Loss: 0.2553, Train Acc: 0.9252 Val Loss: 11.6919, Val Acc: 0.1423
--------------------
Epoch [8/90] Train Loss: 0.2060, Train Acc: 0.9408 Val Loss: 4.6469, Val Acc: 0.2566
--------------------
Epoch [9/90] Train Loss: 0.1854, Train Acc: 0.9463 Val Loss: 1.8986, Val Acc: 0.5637
--------------------
Epoch [10/90] Train Loss: 0.14

###LOADING MODEL

In [ ]:
model_inference = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=8)
model_inference.load_state_dict(torch.load('/content/drive/MyDrive/MatchboxNet_best_model1.pt'))

<All keys matched successfully>

###INFERENCING

In [ ]:
def predict_test_data(model, test_loader):
    device = "cpu"
    model.to(device)
    print("MODEL is using CPU for Inferencing....")
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in tqdm(test_loader, desc="Inferencing"):
            inputs = inputs.to(device)
            inputs = inputs.permute(0, 2, 1)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

test_predictions = predict_test_data(model_inference, test_loader)

MODEL is using CPU for Inferencing....


Inferencing: 100%|██████████| 43/43 [00:49<00:00,  1.16s/it]


###SUBMISSSION

In [ ]:
test['predicted_class'] = train_dataset.label_encoder.inverse_transform(test_predictions)
test[["id" , "class"]] = test[["id" , "predicted_class"]]
test[["id" , "class"]] .to_csv('test_predictions_15__1_.csv', index=False)
test.head()

,id,audio_filepath,duration,predicted_class,class
0,id_u5iqtgjzhx,id_u5iqtgjzhx.wav,2.35800,up,up
1,id_l7ebzcfk5e,id_l7ebzcfk5e.wav,3.22100,down,down
2,id_jbzci8uepl,id_jbzci8uepl.wav,2.94898,yes,yes
3,id_jzil0fw5vs,id_jzil0fw5vs.wav,4.33100,yes,yes
4,id_o7mrvf5wj7,id_o7mrvf5wj7.wav,1.85600,left,left
